# Denison DA210/CS181 Homework 2.g - Step 1

Before you turn this notebook in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells**.

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

In [13]:
import os
import os.path
import pandas as pd

datadir = "publicdata"

---

## Part A: Pivot with one `values` column

**Q1:** Make the following DoL into a `pandas` data frame, assigning it to a variable `df1`.
```
    {'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
     'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
     'baz': [1, 2, 3, 4, 5, 6]}
```

In [14]:
hi =     {'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
     'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
     'baz': [1, 2, 3, 4, 5, 6]}
    
df1 = pd.DataFrame(hi)
# Display the DataFrame
df1

,foo,bar,baz
0,one,A,1
1,one,B,2
2,one,C,3
3,two,A,4
4,two,B,5
5,two,C,6


In [15]:
# Testing cell
assert df1.shape == (6,3)
assert list(df1.columns) == ["foo", "bar", "baz"]
assert list(df1.index) == list(range(6))

**Q2:** Suppose the column `'bar'` should provide the row-label `Index`, the values `'one'` and `'two'` from column `'foo'` should be column labels (so it takes more than one row of `df` to interpret a single observation), and the values themselves should come from the `'baz'` column.  

We can obtain a tidy version of this data using a `pivot` operation.  Perform the `pivot` and assign the result to `df1_tidy`.  Make sure not to modify `df1`.

In [16]:
df1_tidy = df1.pivot(index = "bar", 
                     columns ="foo", 
                     values = "baz")

# Display the DataFrame
df1_tidy

foo,one,two
bar,,
A,1,4
B,2,5
C,3,6


In [17]:
# Testing cell
assert df1_tidy.shape == (3,2)
assert "one" in df1_tidy.columns
assert "two" in df1_tidy.columns
assert df1_tidy.loc["A", "one"] == 1
assert df1_tidy.loc["C", "two"] == 6

**Q3:** Based on the parameters `index`, `columns`, and `values` in the original data frame, how can you tell how many columns and rows will be present after a `pivot` operation?

For pivot operations, we know that the number of rows in the tidied data frame will be the mulplication of the number of unique values in columns and the number of columns of values. There are two unique values in "foo" ("one", "two") and one column for value ("baz"), so we can tell that there would be two columns in the final tidy dataframe. 

---

## Part B: Another dataset with one `values` column

**Q4:** Consider the file `restaurants_gender.csv`, that has aggregated other data and whose rows map from an id, restaurant, and gender to an average rating.  Relative to this aggregation, the data is tidy as it stands.

Load the data from this file into a `DataFrame` named `rest_gender_df`.  You should inspect the file to determine what to use as a row-label `Index`.

In [18]:
df2 = pd.read_csv(os.path.join(datadir, "restaurants_gender.csv"), index_col = "id")
rest_gender_df = pd.DataFrame(df2)


# Display the DataFrame
rest_gender_df

,restaurant,gender,rating
id,,,
1,A,F,82
2,A,M,79
3,B,F,57
4,B,M,68


In [19]:
# Testing cell
assert rest_gender_df.shape == (4,3)

**Q5:** Pivot the `rest_gender_df` data into a matrix presentation with `restaurant` down one axis (as a row-label `Index`) and `gender` across the other axis (as column label `Index`), a form that might make for good presentation.  Store the result as `rest_gender_mat`.

Note that if you end up with a multi-level column `Index`, you can use [`droplevel`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.droplevel.html) to remove the `rating` as the outer level.

In [20]:
rest_gender_mat = rest_gender_df.pivot(index = "restaurant", 
                                       columns = "gender", 
                                       values = "rating")


# Display the matrix DataFrame
rest_gender_mat

gender,F,M
restaurant,,
A,82,79
B,57,68


In [21]:
# Testing cell
assert rest_gender_mat.shape == (2,2)
assert rest_gender_mat.loc["A", "F"] == 82 # requires one-level column Index
assert rest_gender_mat.loc["A", "M"] == 79
assert rest_gender_mat.loc["B", "F"] == 57
assert rest_gender_mat.loc["B", "M"] == 68

---

## Part C: Pivot with multiple columns for `index`

**Q6:** Consider the following version of `indicators`.  In the markdown cell following this code cell, state a tidy-data principle it fails to satisfy and why.

In [22]:
data = [["IND", 2005, "pop",  1147.61],
        ["IND", 2010, "pop",  1234.28],
        ["IND", 2015, "pop",  1310.15],
        ["USA", 2005, "pop",   295.52],
        ["USA", 2010, "pop",   309.33],
        ["USA", 2015, "pop",   320.74],
        ["IND", 2005, "gdp",   820.38],
        ["IND", 2010, "gdp",  1675.62],
        ["IND", 2015, "gdp",  2103.59],
        ["USA", 2005, "gdp", 13036.64],
        ["USA", 2010, "gdp", 14992.05],
        ["USA", 2015, "gdp", 18219.30]]

columns = ["code", "year", "indicator", "value"]

indicators0_untidy = pd.DataFrame(data, columns=columns)

# Display the untidy DataFrame
indicators0_untidy

,code,year,indicator,value
0,IND,2005,pop,1147.61
1,IND,2010,pop,1234.28
2,IND,2015,pop,1310.15
3,USA,2005,pop,295.52
4,USA,2010,pop,309.33
5,USA,2015,pop,320.74
6,IND,2005,gdp,820.38
7,IND,2010,gdp,1675.62
8,IND,2015,gdp,2103.59
9,USA,2005,gdp,13036.64


The dataframe indicators0_untidy fails TD1 because the column "indicator" has variables "pop" and "gdp" in it, not values. This dataframe also fails TD2 because the column "value" contains values for both pop and gdp. 

**Q7:** Use the `pivot_table` operation to pivot this data into a tidy form that includes a multi-level row index.  Store your result in `indicators0_tidy` and do not modify the original `indicators0_untidy`.

Hint: If you find yourself with a multi-level column index after calling `pivot_table`, you may find [`droplevel`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.droplevel.html) helpful to remove the outer level of the column multi-level index.

In [23]:
indicators0_tidy = indicators0_untidy.pivot(index = ["code", "year"], 
                                              columns = "indicator", 
                                              values = "value")

# Display the resulting tidy DataFrame
indicators0_tidy

indicator       gdp      pop
code year                   
IND  2005    820.38  1147.61
     2010   1675.62  1234.28
     2015   2103.59  1310.15
USA  2005  13036.64   295.52
     2010  14992.05   309.33
     2015  18219.30   320.74

In [24]:
# Testing cell
assert indicators0_tidy.shape == (6,2)
assert indicators0_tidy.loc[("IND", 2005), "pop"] == 1147.61
assert indicators0_tidy.loc[("USA", 2015), "gdp"] == 18219.30